In [1]:
import numpy as np
import pandas as pd
import os
import pickle

In [4]:
reviews_by_restaurant = []
restaurant_names = []
for filename in os.listdir('reviews'):
    df = pd.read_csv('reviews/' + filename)
    combined = ' '.join(df['review'].to_list()).lower().strip()
    reviews_by_restaurant.append(combined)
    restaurant_names.append(filename[:-4])
df = pd.DataFrame({'name': restaurant_names, 'text': reviews_by_restaurant})
print(df.head())

                         name  \
0                   10 Scotts   
1  15 Stamford By Alvin Leung   
2                 21 on Rajah   
3                  25 Degrees   
4                       665°F   

                                                text  
0  there were flys when i had deserts course. the...  
1  eel congee was cold, rest of the dishes was ju...  
2  the whole experience was the most absolute per...  
3  we were walking around all day and needed some...  
4  service was terrible. slow unattentive unwelco...  


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df['text'])
print(vectors.shape)
vector_df = pd.DataFrame(vectors.toarray(), columns = [i for i in range(vectors.shape[1])])
print(vector_df.shape)
vector_df['name'] = df['name']

query = 'cable car'
query_vector = vectorizer.transform([query])
similarity_scores = cosine_similarity(query_vector, vectors)

df['similarity_scores'] = similarity_scores[0]
df = df.sort_values(by='similarity_scores', ascending=False)
relevant_results = df['name'][:16]
print(relevant_results)

(429, 35964)
(429, 35964)
14                                 Arbora
103                 Dusk Restaurant & Bar
105        East Coast Lagoon Food Village
210                      Marche Movenpick
129           Fu Lin Men Cantonese Dining
354              The Chop House Vivo City
333                          Straits Cafe
263                             PeraMakan
343    Tamarind Hill Singapore By Samadhi
265                           Pietrasanta
141                                  GRUB
122                                  Flnt
107                                elemen
24                                  Atout
298                           Riders Cafe
179             La Braceria Pizza & Grill
Name: name, dtype: object


In [21]:
vector_df.to_csv('restaurant_vectors.csv', index=False)

In [22]:
out = open('query_vectorizer.pkl', 'wb')
pickle.dump(vectorizer, out)

In [23]:
out.close()

In [26]:
similarity_scores = cosine_similarity(query_vector, vector_df.drop(['name'], axis=1))